In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AAZ852592 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-141603
Azure region: southcentralus
Subscription id: 9b72f9e6-56c5-4c16-991b-19c652994860
Resource group: aml-quickstarts-141603


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
compute_name = "udacity-cluster"
try:
    compute_target = ComputeTarget(workspace=ws, name=compute_name)
    print('Compute cluster already exists.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', min_nodes=0, max_nodes=4)
    compute_target = ComputeTarget.create(ws, compute_name, compute_config)
    print('Create compute cluster...')

compute_target.wait_for_completion(show_output=True)

print(compute_target.get_status().serialize())


Create compute cluster...
Creating....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-03-29T22:09:05.110000+00:00', 'errors': None, 'creationTime': '2021-03-29T22:09:03.051262+00:00', 'modifiedTime': '2021-03-29T22:09:18.581742+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [6]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
# from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
import os

# Specify parameter sampler
# ps = ### YOUR CODE HERE ###
ps = RandomParameterSampling(
    {
        '--C' : uniform(0.5, 1.5),
        '--max_iter': choice(50, 100, 150, 200)
    }
)
# Specify a Policy
# policy = ### YOUR CODE HERE ###
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
# est = ### YOUR CODE HERE ###
est = SKLearn(source_directory = os.path.join( "./" ),
        entry_script="train.py",
        compute_target=compute_target
    )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
# hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,
        primary_metric_name='Accuracy',
        primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
        policy=policy,
        estimator=est,
        max_total_runs=16,
        max_concurrent_runs=4
    )


'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [7]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status() == "Completed")


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_1ac675de-ccfd-40bb-a344-599a99413630
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_1ac675de-ccfd-40bb-a344-599a99413630?wsid=/subscriptions/9b72f9e6-56c5-4c16-991b-19c652994860/resourcegroups/aml-quickstarts-141603/workspaces/quick-starts-ws-141603

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-03-29T22:21:54.187051][API][INFO]Experiment created<END>\n""<START>[2021-03-29T22:21:54.790684][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-03-29T22:21:54.973487][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-03-29T22:21:55.6859739Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_1ac675de-ccfd-40bb-a344-599a99413630
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_1ac675de-ccfd-40bb-a344-599a99413630?wsid=/subscriptions/9b72f

In [9]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
#print("Best run metrics:", best_run.get_metrics())
#print("Best run details:", best_run.get_details())
#print("Best run file names:", best_run.get_file_names())

print('Best Run:', best_run.id)
print('Best Run - Accuracy:', best_run.get_metrics()['Accuracy'])
print('Best Run - Parameters:', best_run.get_details()['runDefinition']['arguments'])

Best Run: HD_1ac675de-ccfd-40bb-a344-599a99413630_0
Best Run - Accuracy: 0.9072837632776934
Best Run - Parameters: ['--C', '0.5221343669159602', '--max_iter', '200']


In [12]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files('https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')


In [13]:
from train import clean_data

# Use the clean_data function to clean your data.
# x, y = clean_data(### YOUR DATA OBJECT HERE ###)
x, y = clean_data(ds)


In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='target',
    n_cross_validations=5)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
remote_run = exp.submit(automl_config, show_output = False)
remote_run.wait_for_completion()


In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)
print("Best run metrics :",best_run.get_metrics())
print("Best run details :",best_run.get_details())
